In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)


label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

data= pd.concat([data.drop("Geography", axis=1),geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [3]:
X= data.drop("Exited", axis=1)
y= data['Exited']

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [5]:
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender =pickle.load(file)
with open('b.pkl','rb') as file:
    b =pickle.load(file)
with open('scaler.pkl','rb') as file:
     scaler =pickle.load(file)


In [6]:
## define a function to create a model and try different parameters(KerasCLassifier)
def create_model(neurons=32,layers=1):
    model= Sequential()
    model.add(Dense(neurons,activation="relu",input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation="relu"))

        model.add(Dense(1,activation="sigmoid"))
        model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['Accuracy'])
        return model

In [7]:
## create a keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [8]:
### Dei=fine the grid seacrch parameters
param_grid= {
    "neurons":[16,32,64,128],
    "layers":[1,2,3],
    "epochs":[50,100]
    }

In [9]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(X_train,y_train)
### Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

KeyboardInterrupt: 